# Coronavirus Brasil - Gráficos interativos
> Gráficos e análises dos estados brasileiros

- toc: false
- branch: master
- badges: false
- comments: false
- author: Vinicius Bastos Gomes & Fernando Wittmann
- categories: [coronavirus]



In [0]:
#hide
# Imports
import warnings
warnings.filterwarnings('ignore')
import gzip
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, timedelta
sns.set(style="ticks", rc={"lines.linewidth": 1.6})
sns.set_palette('dark')

In [0]:
#hide
# Functions
def sigla_nome(x):
    return dic[x]

def populacao(x):
    return popu[x]

In [231]:
#hide

!pip install plotly --upgrade

Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.6.0)


In [232]:
#hide
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [233]:
#hide
# Abrir pasta com os arquivos
%cd "/content/drive/My Drive/CoronaVirus Brasil"

/content/drive/My Drive/CoronaVirus Brasil


In [0]:
#hide
#!rm '/content/drive/My Drive/CoronaVirus Brasil/teste.csv'


In [0]:
#hide Tá NO MANUAL AINDA!
#!wget https://mobileapps.saude.gov.br/esus-vepi/files/unAFkcaNDeXajurGB7LChj8SgQYS2ptm/b7d2591790c7eee6abf3842983a46d92_Download_COVID19_20200404.csv -O teste.csv

In [0]:
#hide
#dictionaries

dic = {'AC': "Acre",  'AL': "Alagoas",  'AM': "Amapá", 'AP': "Amazonas", 'BA': "Bahia", 'CE': "Ceará", 'DF': "Distrito Federal",'ES': "Espírito Santo",'GO': "Goiás",'MA': "Maranhão",'MG': "Minas Gerais",'MS': "Mato Grosso do Sul",'MT': "Mato Grosso",'PA': "Pará",'PB': "Paraíba",'PE': "Pernambuco",'PI': "Piauí",'PR': "Paraná",'RJ': "Rio de Janeiro",'RN': "Rio Grande do Norte",'RO': "Rondônia",'RR': "Roraima",'RS': "Rio Grande do Sul",'SC': "Santa Catarina",'SE': "Sergipe",'SP': "São Paulo",'TO': "Tocantins"} 
popu ={'AC': 881935,'AL': 3337357,'AM': 4144597,'AP': 845731,'BA': 14873064,'CE': 9132078,'DF': 3015268,'ES': 4018650,'GO': 7018354,'MA': 7075181,'MG': 21168791,'MS': 2778986,'MT': 3484466,'PA': 8602865,'PB': 4018127,'PE': 9557071,'PI': 3273227,'PR': 11433957,'RJ': 17264943,'RN': 3506853,'RO': 1777225,'RR': 605761,'RS': 11377239,'SC': 7164788,'SE': 2298696,'SP': 45919049,'TO': 1572866}

In [0]:
#hide
casos = pd.read_csv('teste.csv',sep=';',parse_dates=['date'],dayfirst=True)
casos.columns = ['regiao',	'estado',	'data',	'casosNovos',	'casosAcumulados',	'obitosNovos',	'obitosAcumulados']



In [237]:
#hide
# adding a relative approach column
casos['população']=casos['estado']
casos['população'] = casos['população'].apply(populacao)
casos['Relativa'] = 10000*(casos['casosAcumulados']/casos['população']) #a cada 10 mil
casos.tail(4)

,regiao,estado,data,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados,população,Relativa
1805,Centro-Oeste,DF,2020-04-02,15,370,1,4,3015268,1.227088
1806,Centro-Oeste,DF,2020-04-03,32,402,1,5,3015268,1.333215
1807,Centro-Oeste,DF,2020-04-04,52,454,2,7,3015268,1.505670
1808,Centro-Oeste,DF,2020-04-05,14,468,0,7,3015268,1.552101


In [0]:
#hide
# Change states` names

casos['estado'] = casos['estado'].apply(sigla_nome)


In [239]:
#hide
casos[casos['estado']=='Minas Gerais'].tail(5)

,regiao,estado,data,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados,população,Relativa
1134,Sudeste,Minas Gerais,2020-04-01,39,314,1,3,21168791,0.148332
1135,Sudeste,Minas Gerais,2020-04-02,56,370,1,4,21168791,0.174786
1136,Sudeste,Minas Gerais,2020-04-03,27,397,2,6,21168791,0.187540
1137,Sudeste,Minas Gerais,2020-04-04,33,430,0,6,21168791,0.203129
1138,Sudeste,Minas Gerais,2020-04-05,68,498,0,6,21168791,0.235252


In [0]:
#hide
#manipulating datetime
todei = date.today()
today = todei.strftime("%d-%m-%Y")
today_us = todei.strftime("%Y-%m-%d")
today_us
yesterdei = date.today() - timedelta(days=1)
yesterday = yesterdei.strftime('%Y-%m-%d')

<center> <h2> Gráficos de contaminação </h2> </center>


Casos novos no país.

In [348]:

#hide_input

#df_last.sort_values('Casos')
brasiltotal = casos_mar5.groupby('data').sum()[['casosNovos','obitosNovos']]
brasiltotal['Data']=brasiltotal.index

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(x=brasiltotal['Data'], y=brasiltotal['casosNovos'], name='Casos novos no Brasil'))
fig.add_trace(go.Line(x=brasiltotal['Data'], y=brasiltotal['casosNovos'].rolling(2).mean(), name='Média móvel (2 períodos)'))
fig.add_trace(go.Line(x=brasiltotal['Data'], y=brasiltotal['casosNovos'].rolling(5).mean(), name='Média móvel (5 períodos)'))
                    
fig.update_layout(title='Casos novos no Brasil', xaxis_title='Data',yaxis_title='Casos',legend={'font':{'size': 7}})
fig.update_layout(legend=dict(x=0,y=1,traceorder="normal",font=dict(
    family="sans-serif",size=9,color="black"),bgcolor="rgba(0,0,0,0)"))

Os gráficos a seguir apresentam curvas de contaminação dos estados brasileiros.  É importante notar que as curvas de contaminação estão amplamente relacionadas à quantidade de testes que os países tem realizado.
Até o momento estarão apenas na escala aritmética.

In [241]:
#hide
casos_mar5 = casos[casos['data']>='2020-03-05']
casos_mar5.tail(4)

,regiao,estado,data,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados,população,Relativa
1805,Centro-Oeste,Distrito Federal,2020-04-02,15,370,1,4,3015268,1.227088
1806,Centro-Oeste,Distrito Federal,2020-04-03,32,402,1,5,3015268,1.333215
1807,Centro-Oeste,Distrito Federal,2020-04-04,52,454,2,7,3015268,1.505670
1808,Centro-Oeste,Distrito Federal,2020-04-05,14,468,0,7,3015268,1.552101


In [358]:
#hide_input

import plotly.express as px

fig = px.line(casos_mar5, x="data", y="casosAcumulados", title='COVID-19 - Casos confirmados - {}'.format(today),
              color='estado',color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.update_layout(title='Casos novos no Brasil', xaxis_title='Data',yaxis_title='Casos',legend={'font':{'size': 7}})
fig.update_layout(legend=dict(x=0,y=1,traceorder="normal",font=dict(
    family="sans-serif",size=9,color="black"),bgcolor="rgba(1,10,1,0.05)"))
fig.show()

Essa seção conta com gráficos em que o número de casos é dívidido pela população dos estados para que se tenha uma ideia da proporção de infectados por país.

In [242]:
#hide_input

fig = px.line(casos_mar5[casos_mar5['data']<=today_us], x="data", y="Relativa", title='COVID-19 Brasil \n Crescimento por estado a cada 10 mil habitatnes  {}'.format(today),
              color='estado',color_discrete_sequence=px.colors.qualitative.Alphabet)
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [243]:
#hide
x = casos_mar5['data'].max()
casos_mar5[casos_mar5['data']==x].tail(4)

,regiao,estado,data,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados,população,Relativa
1607,Centro-Oeste,Mato Grosso do Sul,2020-04-05,3,65,0,1,2778986,0.233898
1674,Centro-Oeste,Mato Grosso,2020-04-05,4,60,0,1,3484466,0.172193
1741,Centro-Oeste,Goiás,2020-04-05,12,115,1,3,7018354,0.163856
1808,Centro-Oeste,Distrito Federal,2020-04-05,14,468,0,7,3015268,1.552101


In [244]:
#hide
df_last = casos_mar5[casos_mar5['data']==x]
df_last.tail(3)
df_last.sort_values("casosAcumulados",ascending=False)
df_last.tail(4)

,regiao,estado,data,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados,população,Relativa
1607,Centro-Oeste,Mato Grosso do Sul,2020-04-05,3,65,0,1,2778986,0.233898
1674,Centro-Oeste,Mato Grosso,2020-04-05,4,60,0,1,3484466,0.172193
1741,Centro-Oeste,Goiás,2020-04-05,12,115,1,3,7018354,0.163856
1808,Centro-Oeste,Distrito Federal,2020-04-05,14,468,0,7,3015268,1.552101


O gráfico a seguir é um comparativo de casos totais por estado no Brasil.


In [0]:
#hide_input

df_last.sort_values('casosAcumulados',ascending=False,inplace=True)

fig = px.bar(df_last,x="casosAcumulados", y='estado', color='estado',orientation='h',height=800,
             color_discrete_sequence=px.colors.qualitative.Dark24,title='COVID-19 Brasil \n Casos totais por estado - {}'.format(today))#.update_yaxes(categoryorder='total dscending')

fig.update_layout(showlegend=True)

O gráfico a seguir é um comparativo a proporção de habitantes infectados por estado no Brasil.


In [0]:
#hide_input


#df_last.sort_values('Casos')
df_last.sort_values('Relativa',ascending=False,inplace=True)
fig = px.bar(df_last,x="Relativa", y='estado', color='estado',orientation='h',height=800,
             color_discrete_sequence=px.colors.qualitative.Dark24,title='COVID-19 Brasil \n Casos a cada 10 mil habitantes (por estado) - {}'.format(today))#.update_yaxes(categoryorder='total dscending')

fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})


In [0]:
#hide
casos_mar5 = casos_mar5[casos_mar5['data']>='2020-03-12']

In [246]:
#hide
regioes = casos_mar5.groupby(['data','regiao']).sum()[['casosAcumulados','obitosAcumulados']]
regioes.reset_index(level=[0,1],inplace=True)
regioes.tail(3)

,data,regiao,casosAcumulados,obitosAcumulados
122,2020-04-05,Norte,651,19
123,2020-04-05,Sudeste,6678,351
124,2020-04-05,Sul,1213,26


In [0]:
#hide
sul = casos_mar5[(casos_mar5['estado']=='Paraná') | (casos_mar5['estado']=='Santa Catarina') | (casos_mar5['estado']=='Rio Grande do Sul')]

In [0]:
#hide
sudeste = casos_mar5[(casos_mar5['estado']=='São Paulo') | (casos_mar5['estado']=='Minas Gerais') | (casos_mar5['estado']=='Rio de Janeiro') | (casos_mar5['estado']=='Espírito Santo ')]

In [0]:
#hide
centroeste =  casos_mar5[(casos_mar5['estado']=='Goiás') | (casos_mar5['estado']=='Mato Grosso') | (casos_mar5['estado']=='Mato Grosso do Sul')| (casos_mar5['estado']=='Distrito Federal')]

In [0]:
#hide
norte = casos_mar5[(casos_mar5['estado']=='Acre') | (casos_mar5['estado']=='Amapá') | (casos_mar5['estado']=='Amazonas') | (casos_mar5['estado']=='Pará') | (casos_mar5['estado']=='Rondônia')| (casos_mar5['estado']=='Roraima')| (casos_mar5['estado']=='Tocantins')]

In [0]:
#hide
nordeste = casos_mar5[(casos_mar5['estado']=='Alagoas') | (casos_mar5['estado']=='Bahia') | (casos_mar5['estado']=='Ceará') | (casos_mar5['estado']=='Maranhão') | (casos_mar5['estado']=='Paraíba')| (casos_mar5['estado']=='Pernambuco')| (casos_mar5['estado']=='Piauí')| (casos_mar5['estado']=='Rio Grande do Norte')| (casos_mar5['estado']=='Piauí')]

<center> <h2> Curvas de contaminação por regiões</h2> </center>


Comparações entre as regiões do Brasil. Primeiramente uma soma dos casos por regiões e, posteriormente, uma comparação entre os estados das regiões.



In [252]:
#hide_input
fig = px.line(regioes[regioes['data']>='2020-03-05'], x="data", y="casosAcumulados", title='COVID-19 Brasil - Sudeste - {}'.format(today),
              color='regiao',color_discrete_sequence=[ "blue",'green',"darkmagenta","gray","brown","orange","magenta",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [253]:
#hide_input


fig = px.line(sudeste, x="data", y="casosAcumulados", title='COVID-19 Brasil - Sudeste - {}'.format(today),
              color='estado',color_discrete_sequence=[ "blue",'green',"darkmagenta","gray","brown","orange","magenta",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [254]:
#hide_input
#sudeste relativa
#sudeste.sort_values('Relativa',ascending=False,inplace=True)
#plt.figure(figsize=(15,8))
#sns.lineplot(x='Dia',y='Relativa',data=sudeste,hue='Estado',size="Estado")
#plt.title('COVID-19 Brasil - Sudeste - {}\n Contaminados a cada 10 mil habitantes'.format(today))
#plt.grid(color='black', linestyle='--', linewidth=0.17)
#plt.xlabel('Data')
#plt.ylabel('Contaminação relativa')
#plt.savefig("plots/19_sudeste_relativa.png",bbox_inches = 'tight')

fig = px.line(sudeste, x="data", y="Relativa", title='COVID-19 Brasil - Sudeste - {}\n Contaminados a cada 10 mil habitantes'.format(today),
              color='estado',color_discrete_sequence=[ "blue",'green',"darkmagenta","gray","brown","orange","magenta",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [255]:
#hide_input
#sul absoluta
#sul.sort_values('Casos',ascending=False,inplace=True)
#plt.figure(figsize=(15,8))
#sns.lineplot(x='Dia',y='Casos',data=sul,hue='Estado',size="Estado")
#plt.title('COVID-19 Brasil - Sul - {}'.format(today))
#plt.grid(color='black', linestyle='--', linewidth=0.17)
#plt.xlabel('Data')
#plt.ylabel('Casos')
#plt.savefig("plots/20_sul_absoluta.png",bbox_inches = 'tight')

fig = px.line(sul, x="data", y="casosAcumulados", title='COVID-19 Brasil - Sul - {}'.format(today),
              color='estado',color_discrete_sequence=[ "blue",'green',"darkmagenta","gray","brown","orange","magenta",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [256]:
#hide_input
#sul relativa
#sul.sort_values('Relativa',ascending=False,inplace=True)
#plt.figure(figsize=(15,8))
#sns.lineplot(x='Dia',y='Relativa',data=sul,hue='Estado',size="Estado")
#plt.title('COVID-19 Brasil - Sul - {}\n Contaminados a cada 10 mil habitantes'.format(today))
#plt.grid(color='black', linestyle='--', linewidth=0.17)
#plt.xlabel('Data')
#plt.ylabel('Contaminação relativa')
#plt.savefig("plots/21_sul_relativa.png",bbox_inches = 'tight')

fig = px.line(sul, x="data", y="Relativa", title='COVID-19 Brasil - Sul - {}\n Contaminados a cada 10 mil habitantes'.format(today),
              color='estado',color_discrete_sequence=[ "blue",'green',"darkmagenta","gray","brown","orange","magenta",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [257]:
#hide_input
#nordeste absoluta
#nordeste.sort_values('Casos',ascending=False,inplace=True)
#plt.figure(figsize=(15,8))
#sns.lineplot(x='Dia',y='Casos',data=nordeste,hue='Estado',size="Estado")
#plt.title('COVID-19 Brasil - Nordeste - {}'.format(today))
#plt.grid(color='black', linestyle='--', linewidth=0.17)
#plt.xlabel('Data')
#plt.ylabel('Casos')
#plt.savefig("plots/22_nordeste_absoluta.png",bbox_inches = 'tight')
fig = px.line(nordeste, x="data", y="casosAcumulados", title='COVID-19 Brasil - Nordeste - {}'.format(today),
              color='estado',color_discrete_sequence=[ "blue",'green',"darkmagenta","gray","brown","orange",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [258]:
#hide_input
#nordeste relativa
#nordeste.sort_values('Relativa',ascending=False,inplace=True)
#plt.figure(figsize=(15,8))
#sns.lineplot(x='Dia',y='Relativa',data=nordeste,hue='Estado',size="Estado")
#plt.title('COVID-19 Brasil - Nordeste - {}\n Contaminados a cada 10 mil habitantes'.format(today))
#plt.grid(color='black', linestyle='--', linewidth=0.17)
#plt.xlabel('Data')
#plt.ylabel('Contaminação relativa')
#plt.savefig("plots/23_brasil_curva_relativa.png",bbox_inches = 'tight')


fig = px.line(nordeste, x="data", y="Relativa", title='COVID-19 Brasil - Nordeste - {}\n Contaminados a cada 10 mil habitantes'.format(today),
              color='estado',color_discrete_sequence=[ "blue",'green',"darkmagenta","gray","brown","orange",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [259]:
#hide_input
#norte absoluta
#norte.sort_values('Casos',ascending=False,inplace=True)
#plt.figure(figsize=(15,8))
#sns.lineplot(x='Dia',y='Casos',data=norte,hue='Estado',size="Estado")
#plt.title('COVID-19 Brasil - Norte - {}'.format(today))
#plt.grid(color='black', linestyle='--', linewidth=0.17)
#plt.xlabel('Data')
#plt.ylabel('Casos')
#plt.savefig("plots/24_norte_absoluta.png",bbox_inches = 'tight')

fig = px.line(norte, x="data", y="casosAcumulados", title='COVID-19 Brasil - Norte - {}'.format(today),
              color='estado',color_discrete_sequence=[ "blue",'green',"darkmagenta","gray","orange",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [260]:
#hide_input
#norte relativa
#norte.sort_values('Relativa',ascending=False,inplace=True)
#plt.figure(figsize=(15,8))
#sns.lineplot(x='Dia',y='Relativa',data=norte,hue='Estado',size="Estado")
#plt.title('COVID-19 Brasil - Norte - {}\n Contaminados a cada 10 mil habitantes'.format(today))
#plt.grid(color='black', linestyle='--', linewidth=0.17)
#plt.xlabel('Data')
#plt.ylabel('Contaminação relativa')
#plt.savefig("plots/25_norte_relativa.png",bbox_inches = 'tight')
fig = px.line(norte, x="data", y="Relativa", title='COVID-19 Brasil - Norte - {}\n Contaminados a cada 10 mil habitantes'.format(today),
              color='estado',color_discrete_sequence=[ "blue",'green',"darkmagenta","gray","orange",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [261]:
#hide_input
#centroste absoluta
#plt.figure(figsize=(15,8))
#sns.lineplot(x='Dia',y='Casos',data=centroeste,hue='Estado',size="Estado")
#plt.title('COVID-19 Brasil - Centro Oeste - {}'.format(today))
#plt.grid(color='black', linestyle='--', linewidth=0.17)
#plt.xlabel('Data')
#plt.ylabel('Casos')
#plt.savefig("plots/26_centroeste_absoluta.png",bbox_inches = 'tight')

fig = px.line(centroeste, x="data", y="casosAcumulados", title='COVID-19 Brasil - Centro Oeste - {}'.format(today),
              color='estado',color_discrete_sequence=[ 'green',"darkmagenta","orange",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [262]:
#hide_input
#centroeste relativa
#centroeste.sort_values('Relativa',ascending=False,inplace=True)
#plt.figure(figsize=(15,8))
#sns.lineplot(x='Dia',y='Relativa',data=centroeste,hue='Estado',size="Estado")
#plt.title('COVID-19 Brasil - Centro Oeste - {}\n Contaminados a cada 10 mil habitantes'.format(today))
#plt.grid(color='black', linestyle='--', linewidth=0.17)
#plt.xlabel('Data')
#plt.ylabel('Contaminação relativa')
#plt.savefig("plots/27_centroeste_relativa.png",bbox_inches = 'tight')

fig = px.line(centroeste, x="data", y="Relativa", title='COVID-19 Brasil - Centro Oeste - {}\n Contaminados a cada 10 mil habitantes'.format(today),
              color='estado',color_discrete_sequence=[ 'green',"darkmagenta","orange",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

<center> <h2> Óbitos </h2> </center>


In [333]:

#hide_input

#df_last.sort_values('Casos')
brasiltotal = casos_mar5.groupby('data').sum()[['casosNovos','obitosNovos']]
brasiltotal['Data']=brasiltotal.index

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Bar(x=brasiltotal['Data'], y=brasiltotal['obitosNovos'], name='Casos novos',textfont={'size': 2}))
fig.add_trace(go.Line(x=brasiltotal['Data'], y=brasiltotal['obitosNovos'].rolling(2).mean(), name='Média móvel (2 p)',textfont={'size': 2}))
fig.add_trace(go.Line(x=brasiltotal['Data'], y=brasiltotal['obitosNovos'].rolling(5).mean(), name='Média móvel (5 p)',textfont={'size': 2}))
                    


fig.update_layout(title='Óbitos novos no Brasil',
                   xaxis_title='Data',
                   yaxis_title='Óbitos',legend={'font':{'size': 7}})

Comparações entre as regiões do Brasil. Primeiramente uma soma dos casos por regiões e, posteriormente, uma comparação entre os estados das regiões.

In [266]:
#hide_input


fig = px.line(casos_mar5, x="data", y="obitosAcumulados", title='COVID-19 Brasil \n Linhas de óbitos por estado - {} '.format(today),
              color='estado',color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()


<center> <h2> Óbitos por regiões </h2> </center>


In [267]:
#hide_input

fig = px.line(regioes[regioes['data']>='2020-03-05'], x="data", y="obitosAcumulados", title='COVID-19 Brasil - Sudeste - {}'.format(today),
              color='regiao',color_discrete_sequence=[ "blue",'green',"darkmagenta","gray","brown","orange","magenta",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [268]:
#hide_input


fig = px.line(sudeste, x="data", y="obitosAcumulados", title='COVID-19 Brasil (Óbitos) - Sudeste - {}'.format(today),
              color='estado',color_discrete_sequence=[ 'green',"darkmagenta","orange",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [269]:
#hide_input
#Sul absoluta

#sul.sort_values('Mortes',ascending=False,inplace=True)
#plt.figure(figsize=(15,8))
#sns.lineplot(x='Dia',y='Mortes',data=sul,hue='Estado',size="Estado")
#plt.title('COVID-19 Brasil (Óbitos) - Sul - {}'.format(today))
#plt.grid(color='black', linestyle='--', linewidth=0.17)
#plt.xlabel('Data')
#plt.ylabel('Óbitos')
#plt.savefig("plots/18_sudeste_absoluta.png",bbox_inches = 'tight')

fig = px.line(sul, x="data", y="obitosAcumulados", title='COVID-19 Brasil (Óbitos) - Sul - {}'.format(today),
              color='estado',color_discrete_sequence=[ 'green',"darkmagenta","orange",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [270]:
#hide_input
#nordeste absoluta

#nordeste.sort_values('Mortes',ascending=False,inplace=True)
#plt.figure(figsize=(15,8))
#sns.lineplot(x='Dia',y='Mortes',data=nordeste,hue='Estado',size="Estado")
#plt.title('COVID-19 Brasil (Óbitos) - Nordeste - {}'.format(today))
#plt.grid(color='black', linestyle='--', linewidth=0.17)
#plt.xlabel('Data')
#plt.ylabel('Óbitos')
#plt.savefig("plots/18_sudeste_absoluta.png",bbox_inches = 'tight')

fig = px.line(nordeste, x="data", y="obitosAcumulados", title='COVID-19 Brasil (Óbitos) - Nordeste - {}'.format(today),
              color='estado',color_discrete_sequence=[ 'green',"darkmagenta","orange",'red',"darkblue","lightgreen",'coral','black'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [271]:
#hide_input
#centroeste absoluta

#centroeste.sort_values('Mortes',ascending=False,inplace=True)
#plt.figure(figsize=(15,8))
#sns.lineplot(x='Dia',y='Mortes',data=centroeste,hue='Estado',size="Estado")
#plt.title('COVID-19 Brasil (Óbitos) - Centro Oeste - {}'.format(today))
#plt.grid(color='black', linestyle='--', linewidth=0.17)
#plt.xlabel('Data')
#plt.ylabel('Óbitos')
#plt.savefig("plots/18_sudeste_absoluta.png",bbox_inches = 'tight')

fig = px.line(centroeste, x="data", y="obitosAcumulados", title='COVID-19 Brasil (Óbitos) - Centro Oeste - {}'.format(today),
              color='estado',color_discrete_sequence=[ 'green',"darkmagenta","orange",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [272]:
#hide_input
#norte absoluta

#nordeste.sort_values('Mortes',ascending=False,inplace=True)
#plt.figure(figsize=(15,8))
#sns.lineplot(x='Dia',y='Mortes',data=norte,hue='Estado',size="Estado")
#plt.title('COVID-19 Brasil (Óbitos) - Norte - {}'.format(today))
#plt.grid(color='black', linestyle='--', linewidth=0.17)
#plt.xlabel('Data')
#plt.ylabel('Óbitos')
#plt.savefig("plots/18_sudeste_absoluta.png",bbox_inches = 'tight')

fig = px.line(norte, x="data", y="obitosAcumulados", title='COVID-19 Brasil (Óbitos) - Norte - {}'.format(today),
              color='estado',color_discrete_sequence=[ 'green',"darkmagenta","orange",'red',"darkblue","lightgreen",'coral','lightblue'])
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [0]:
#hide
!rm '/content/drive/My Drive/CoronaVirus Brasil/caso.csv.gz'

In [0]:
#hide
# Parameters
BOLETIM = 'https://data.brasil.io/dataset/covid19/boletim.csv.gz'
CASOS = 'https://data.brasil.io/dataset/covid19/caso.csv.gz'

In [275]:
#hide
# Execution
!wget {CASOS}

--2020-04-06 14:01:31--  https://data.brasil.io/dataset/covid19/caso.csv.gz
Resolving data.brasil.io (data.brasil.io)... 192.99.5.40
Connecting to data.brasil.io (data.brasil.io)|192.99.5.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63815 (62K) [application/gzip]
Saving to: ‘caso.csv.gz’

caso.csv.gz         100%[===================>]  62.32K  --.-KB/s    in 0.02s   

2020-04-06 14:01:32 (2.58 MB/s) - ‘caso.csv.gz’ saved [63815/63815]



In [0]:
#hide
with gzip.open('caso.csv.gz', 'rb') as f_in:
    with open('caso.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [277]:
#hide
casos = pd.read_csv('caso.csv')
casos.tail(4)

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
6380,2020-03-20,AP,Macapá,city,1,0.0,1,False,503327.0,1600303.0,0.19868,NaN
6381,2020-04-05,AP,Oiapoque,city,1,0.0,2,True,27270.0,1600501.0,3.66703,NaN
6382,2020-04-04,AP,Oiapoque,city,1,0.0,1,False,27270.0,1600501.0,3.66703,NaN
6383,2020-04-05,AP,Santana,city,2,0.0,1,True,121364.0,1600600.0,1.64794,NaN


In [278]:
#hide
casos_por_cidade = casos.groupby(['state','city','date']).sum()[['confirmed', 'deaths']]
casos_por_cidade = casos_por_cidade.reset_index(level=[0,1])

casos_por_cidade.head(5)


,state,city,confirmed,deaths
date,,,,
2020-03-29,AC,Acrelândia,2,0.0
2020-03-30,AC,Acrelândia,6,0.0
2020-03-31,AC,Acrelândia,7,0.0
2020-04-01,AC,Acrelândia,8,0.0
2020-04-02,AC,Acrelândia,8,0.0


In [0]:
#hide
cidade_last = casos_por_cidade[casos_por_cidade.index==str(yesterday)]


In [0]:
#hide
cap = ['Rio Branco','Maceió','Macapá','Manaus','Salvador','Fortaleza','Brasília','Vitória','Goiânia','São Luís','Cuiabá','Campo Grande','Belo Horizonte','Belém','João Pessoa','Curitiba','Recife','Teresina','Rio de Janeiro','Natal','Porto Alegre','Porto Velho','Boa Vista','Florianópolis','São Paulo','Aracaju','Palmas']

In [281]:
#hide
casos_por_cidade.columns

Index(['state', 'city', 'confirmed', 'deaths'], dtype='object')

In [0]:
#hide
capitais = casos_por_cidade[casos_por_cidade['city'].isin(['Rio Branco','Maceió','Macapá','Manaus','Salvador','Fortaleza','Brasília','Vitória','Goiânia','São Luís','Cuiabá','Campo Grande','Belo Horizonte','Belém','João Pessoa','Curitiba','Recife','Teresina','Rio de Janeiro','Natal','Porto Alegre','Porto Velho','Boa Vista','Florianópolis','São Paulo','Aracaju','Palmas'])]
capitais_15 = capitais[capitais.index>='2020-03-20']

In [283]:
#hide
capitais_15.sort_values('confirmed',ascending=False,inplace=True)
teste = capitais_15.groupby(capitais_15.index)
teste

In [285]:
#hide
pro_bra = casos.groupby('date').sum()[['deaths','confirmed']]
pro_bra.tail(4)

,deaths,confirmed
date,,
2020-04-02,589.0,16196
2020-04-03,730.0,18424
2020-04-04,883.0,20552
2020-04-05,909.0,20608


In [0]:
#hide
pro_bra['Alemanha'] = pro_bra['deaths']*100
pro_bra['Coreia'] = pro_bra['deaths']*166.6
pro_bra['Data'] = pro_bra.index
pro_bra.columns = ['Óbitos','Brasil registrado','Projeção Alemanha','Projeção Coréia do Sul','Data']
pro_bra = pro_bra[pro_bra['Data']>='2020-03-17']
pro_bra.tail(3)

,Óbitos,Brasil registrado,Projeção Alemanha,Projeção Coréia do Sul,Data
date,,,,,
2020-04-03,730.0,18424,73000.0,121618.0,2020-04-03
2020-04-04,883.0,20552,88300.0,147107.8,2020-04-04
2020-04-05,909.0,20608,90900.0,151439.4,2020-04-05


<center> <h2> Capitais brasileiras </h2> </center>



Comparações entre as capitais de estadis do Brasil

In [0]:
#hide

capitais['Data'] = capitais.index
capitais.sort_values(['state','Data'])
capitais.columns = ['Estado','Cidade','Casos','Óbitos','Data']


In [287]:
#hide
fig = px.line(capitais[capitais['Data']>='2020-03-15'], x="Data", y="Casos", title='COVID-19 Capitais Brasileiras - {}'.format(today),
              color='Cidade',color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

In [288]:
#hide_input
fig = px.line(capitais[capitais['Data']>='2020-03-15'], x="Data", y="Óbitos", title='COVID-19 Capitais Brasileiras - Óbitos - {}'.format(today),
              color='Cidade',color_discrete_sequence=px.colors.qualitative.Dark24)
fig.update_layout(legend= {'borderwidth':0,'orientation':'v','yanchor':'top','font':{'size':8}})
fig.show()

<center> <h2> 200 cidades com maiores números de casos </h2> </center>


In [0]:
#hide_input
pd.set_option('display.max_rows', None) 
x= cidade_last.sort_values('confirmed',axis=0,ascending=False)
x.columns = ['Estado','Cidade','Casos','Mortes']
x.reset_index(level=[0],drop=True)
x.head(200)

,Estado,Cidade,Casos,Mortes
date,,,,
2020-04-05,SP,São Paulo,3612,220.0
2020-04-05,RJ,Rio de Janeiro,1068,42.0
2020-04-05,DF,Brasília,468,7.0
2020-04-05,AM,Manaus,379,11.0
2020-04-05,MG,Belo Horizonte,262,3.0
2020-04-05,RS,Porto Alegre,254,5.0
2020-04-05,BA,Salvador,244,7.0
2020-04-05,PR,Curitiba,172,0.0
2020-04-05,PE,Recife,139,14.0


In [290]:
casos_mar5.columns

Index(['regiao', 'estado', 'data', 'casosNovos', 'casosAcumulados',
       'obitosNovos', 'obitosAcumulados', 'população', 'Relativa'],
      dtype='object')